In [1]:
# main file for Project 2- ETL
# KJ, Britt, Jonathan, Stacey

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Extract Data

In [2]:
# Bring in data sets
features_data_set = "Resources/Features data set.csv"
sales_data_set = "Resources/sales data-set.csv"
stores_data_set = "Resources/stores data-set.csv"
bitcoin_data_set = "Resources/Bitcoin Historical Data.csv"

# Transform Data

### Bitcoin DataFrame

In [13]:
# Create a dataframe
df_bitcoin_orig = pd.read_csv(bitcoin_data_set)
df_bitcoin_orig.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,"Jan 01, 2011",0.3,0.3,0.3,0.3,2.82K,0.00%
1,"Jan 02, 2011",0.3,0.3,0.3,0.3,5.35K,0.00%
2,"Jan 03, 2011",0.3,0.3,0.3,0.3,1.43K,0.00%
3,"Jan 04, 2011",0.3,0.3,0.3,0.3,1.88K,0.00%
4,"Jan 05, 2011",0.3,0.3,0.3,0.3,0.36K,0.00%


In [14]:
# Format the date 
df_bitcoin_orig.Date = pd.to_datetime(df_bitcoin_orig.Date, format='%b %d, %Y', dayfirst=True)
df_bitcoin_orig.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2011-01-01,0.3,0.3,0.3,0.3,2.82K,0.00%
1,2011-01-02,0.3,0.3,0.3,0.3,5.35K,0.00%
2,2011-01-03,0.3,0.3,0.3,0.3,1.43K,0.00%
3,2011-01-04,0.3,0.3,0.3,0.3,1.88K,0.00%
4,2011-01-05,0.3,0.3,0.3,0.3,0.36K,0.00%


In [15]:
# Drop unneeded columns
df_bitcoin = df_bitcoin_orig.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis=1)
df_bitcoin.head()

,Date,Price
0,2011-01-01,0.3
1,2011-01-02,0.3
2,2011-01-03,0.3
3,2011-01-04,0.3
4,2011-01-05,0.3


In [16]:
# Create rolling 7-day average
df_bitcoin['bitcoin_price'] = df_bitcoin.iloc[:,1].rolling(window=7).mean()
df_bitcoin.head(14)

,Date,Price,bitcoin_price
0,2011-01-01,0.3,NaN
1,2011-01-02,0.3,NaN
2,2011-01-03,0.3,NaN
3,2011-01-04,0.3,NaN
4,2011-01-05,0.3,NaN
5,2011-01-06,0.3,NaN
6,2011-01-07,0.3,0.300000
7,2011-01-08,0.3,0.300000
8,2011-01-09,0.3,0.300000
9,2011-01-10,0.3,0.300000


In [17]:
# Drop price column
df_bitcoin = df_bitcoin.drop(['Price'], axis=1)
df_bitcoin

,Date,bitcoin_price
0,2011-01-01,NaN
1,2011-01-02,NaN
2,2011-01-03,NaN
3,2011-01-04,NaN
4,2011-01-05,NaN
...,...,...
360,2011-12-27,4.000000
361,2011-12-28,4.042857
362,2011-12-29,4.085714
363,2011-12-30,4.128571


### Features DataFrame

In [18]:
# Create a dataframe
df_features_orig = pd.read_csv(features_data_set)
df_features_orig.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [19]:
# Convert date column
df_features_orig.Date = pd.to_datetime(df_features_orig.Date, format='%d/%m/%Y', dayfirst=True)
df_features_orig.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [20]:
# Merge Bitcoin data for dates present in Features dataframes
df_features = df_features_orig.merge(df_bitcoin, how='inner', on='Date')
df_features
# Can see it added the bitcoin data for each date for each of the 45 stores

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,bitcoin_price
0,1,2011-01-07,48.27,2.976,NaN,NaN,NaN,NaN,NaN,211.404742,7.742,False,0.300000
1,2,2011-01-07,44.69,2.976,NaN,NaN,NaN,NaN,NaN,211.064888,8.028,False,0.300000
2,3,2011-01-07,53.35,2.976,NaN,NaN,NaN,NaN,NaN,214.695510,7.551,False,0.300000
3,4,2011-01-07,39.34,2.980,NaN,NaN,NaN,NaN,NaN,127.191774,6.510,False,0.300000
4,5,2011-01-07,48.30,2.976,NaN,NaN,NaN,NaN,NaN,211.956031,6.634,False,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,41,2011-12-30,34.12,3.119,5251.98,56106.20,134.70,1059.49,2366.54,196.358610,6.759,True,4.128571
2336,42,2011-12-30,48.92,3.428,242.41,1447.26,3.64,183.96,459.06,130.071032,7.874,True,4.128571
2337,43,2011-12-30,41.83,3.129,415.59,1926.05,NaN,50.66,837.74,211.074055,10.148,True,4.128571
2338,44,2011-12-30,31.53,3.119,409.92,1566.42,NaN,17.64,153.90,130.071032,6.078,True,4.128571


In [ ]:
# Delete columns, rename columns, convert f to c for a new column?
# Could also create a column for which week of the year it is based on the date?

### Sales DataFrame

In [11]:
# Create a dataframe
df_sales = pd.read_csv(sales_data_set)
df_sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


In [12]:
# Convert date column
df_sales.Date = pd.to_datetime(df_sales.Date, format='%d/%m/%Y', dayfirst=True)
df_sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [ ]:
# add promotional/markdown to sales df as boolean (i.e., if a markdown exist == TRUE, if NAN == FALSE)

### Stores DataFrame

In [ ]:
# Create a dataframe
df_stores = pd.read_csv(stores_data_set)
df_stores.head()

# Load Data